In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from groq import Groq
from google.colab import userdata

api = userdata.get('GROQ_API_KEY')
client = Groq(api_key=api)

In [ ]:
df = pd.read_csv('/content/sample_20_rows.csv')
X = df.drop('Label', axis=1)
y = df['Label']

In [ ]:
Few shot example = """
Example :

Input :
{Network logs}
Output :
{Analysis}
"""
system_message = """
You're a cybersecurity expert classifying network flow logs as BENIGN or DDOS.
Use ReAct (Reason + Act):
1. REASON: Analyze IPs, ports, protocols, duration, packets, byte rates.
2. ACT: Identify patterns or anomalies to classify.
Be clear and concise. End with: "Final Answer: BENIGN" or "Final Answer: DDOS".
For answering use the following template {few shot example}
"""

In [ ]:
system_message = """
You are a cybersecurity expert trained to analyze network flow logs and determine whether the traffic is benign or part of a Distributed Denial of Service (DDoS) attack.

Use the ReAct (Reasoning + Acting) approach:
1. First, REASON through the meaning of the network features such as IPs, ports, protocols, flow duration, packet counts, and byte rates.
2. Then, ACT by concluding whether the flow is BENIGN or DDOS based on patterns, anomalies, or suspicious behavior.

Be concise but clear in your reasoning, and always end with the final classification in the format: "Final Answer: BENIGN" or "Final Answer: DDOS".
"""

few_shot_example = """
Example:

Input:
Source IP : 172.16.0.5
Destination IP : 149.171.126.12
Source Port : 49855
Destination Port : 443
Protocol : 6
Flow Duration : 1147838
Total Fwd Packets : 12
Total Backward Packets : 15
Flow Bytes/s : 12.2

Analysis:
Let me analyze this network flow:
- Source IP 172.16.0.5 is a private IP address (likely internal network)
- Destination IP 149.171.126.12 is a public IP
- Source Port 49855 is a high ephemeral port (normal for client connections)
- Destination Port 443 indicates HTTPS traffic (normal web browsing)
- Protocol 6 is TCP (standard for web browsing)
- Flow Duration of 1147838 microseconds (~1.15 seconds) is reasonable for a web request
- Packet counts (12 forward, 15 backward) are balanced and typical for normal web traffic
- Flow Bytes/s at 12.2 is relatively low and consistent with regular web browsing

This appears to be normal HTTPS web traffic from an internal network to an external server with balanced packet exchange and reasonable data rates.

Final Answer: BENIGN
"""

In [ ]:
for i in range(20):
    prompt = f"""
Source IP : {X['Source IP'][i]}
Destination IP : {X['Destination IP'][i]}
Source Port : {X['Source Port'][i]}
Destination Port : {X['Destination Port'][i]}
Protocol : {X['Protocol'][i]}
Flow Duration : {X['Flow Duration'][i]}
Total Fwd Packets : {X['Total Fwd Packets'][i]}
Total Backward Packets : {X['Total Backward Packets'][i]}
Flow Bytes/s : {X['Flow Bytes/s'][i]}
"""

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": few_shot_example
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="llama-3.1-8b-instant",
        temperature=0.5,
        max_completion_tokens=1024,
        top_p=0.5,
        stream=False
    )

    print(f"\nResponse for index {i}:")
    print(response.choices[0].message.content)


Response for index 0:
Let me analyze this network flow:

- Source IP 172.16.0.5 is a private IP address (likely internal network)
- Destination IP 192.168.50.1 is a private IP address (likely internal network)
- Source Port 842 is a relatively low port number (not typical for client connections)
- Destination Port 21484 is an ephemeral port (normal for server connections)
- Protocol 17 is UDP (not standard for web browsing, but can be used for various services)
- Flow Duration of -0.1944709240047926 seconds is negative, which is unusual and likely indicates a miscount or error
- Packet counts (0.3251184533735609 forward, -0.1451238114943991 backward) are not balanced and seem to be incorrect
- Flow Bytes/s at -0.4163960526841666 is negative, which is not possible and likely indicates an error

This flow appears to be anomalous due to the negative flow duration, packet counts, and byte rates, which are likely errors in the flow log. Additionally, the source and destination ports and pr